In [1]:
import pandas as pd
import plotly.express as px
# import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.set_option('display.max_rows',None)

In [2]:
def loadCleanData(path:str):
    # read data
    data = pd.read_csv(path)
    
    # drop columns
    data = data[['Market', 'TrainNumber', 'DepartureDateTime', 'ArrivalDateTime','Origin', 'Destination', 'Revenue', 'Bookings','CommercialClass']]
    
    # change date format
    data['DepartureDateTime'] = pd.to_datetime(data['DepartureDateTime'] )
    data['ArrivalDateTime'] = pd.to_datetime(data['ArrivalDateTime'] )

    # index data
    cleanData = data.groupby(by=['Market', 'TrainNumber', 'DepartureDateTime', 'CommercialClass'])[['Revenue','Bookings']].sum()
    cleanData = cleanData.reset_index()

    return cleanData, data


def rule_80_20(df:pd.DataFrame, attr:str, index_1=0, index_2=0, value_index_1=0, value_index_2=0, percent = 80.0):

    if index_1 == 0 and index_2 == 0:
        index_1 = 'Year'
        index_2 = 'MonthYear'

    marketFor = pd.pivot_table(df, index=[index_1, index_2, 'Market'], values=attr, aggfunc='sum').sort_values(by=[index_1, index_2, attr], ascending=[True, True, False])
    marketFor = marketFor.div(marketFor.groupby([index_1, index_2]).transform('sum')) * 100

    sumAccumulation = marketFor.groupby([index_1, index_2]).cumsum()
    filterIndex = sumAccumulation[attr] <= percent
    indexFilterIndex = filterIndex[filterIndex].index

    marketFor_80 = marketFor.loc[indexFilterIndex].reset_index()
    marketFor_80.columns = [index_1, index_2, 'Market',	'PercentRevenue']
    numberMarketFor_80 = marketFor_80.reset_index().groupby([index_1, index_2])['Market'].nunique().reset_index()
    numberMarketFor_80.columns = [index_1, index_2,	'NumMarket']

    if value_index_1 != 0:
        marketFor_80 = marketFor_80[marketFor_80[index_1]==value_index_1]
        numberMarketFor_80 = numberMarketFor_80[numberMarketFor_80[index_1]==value_index_1]

    if value_index_2 != 0:
        marketFor_80 = marketFor_80[marketFor_80[index_2]==value_index_2]
        numberMarketFor_80 = numberMarketFor_80[numberMarketFor_80[index_2]==value_index_2]

    if value_index_1 != 0 and value_index_2 != 0:
        return marketFor_80, numberMarketFor_80['NumMarket'][1]
    else:
        return marketFor_80, numberMarketFor_80
    

def addTimeFeatures(df: pd.DataFrame, attr: str):
    df['DepatureDate'] = df[attr].dt.date
    df['DepatureTime'] = df[attr].dt.time
    df['HourDay'] = df[attr].dt.hour
    df['DayWeek'] = df[attr].dt.day_name()
    df['WeekYear'] = df[attr].dt.isocalendar().week
    df['MonthYear'] = df[attr].dt.month
    df['Year'] = df[attr].dt.year
    
    return df

In [3]:
# cleanData, data = loadCleanData('/home/wilmer/Documentos/Codes/WORKSHOP/data/anonymizedHistoricalData.csv')
cleanData, data = loadCleanData('C:/Tesis_wilmer/workshop/data/anonymizedHistoricalData.csv')

In [21]:
# Adding temporality
cleanData = addTimeFeatures(cleanData, 'DepartureDateTime')

# Number of Markets
numTotalMarket = cleanData['Market'].nunique()
numMarketMonth = cleanData.groupby(by=['Year', 'MonthYear'])['Market'].nunique().reset_index()

# Rule 80-20
marketForDemand, numberMarketForDemand = rule_80_20(df = cleanData, 
                                                    attr = 'Bookings', 
                                                    # value_index_1 = 2023,
                                                    # value_index_2 = 1
                                                    )

marketForRevenue, numberMarketForRevenue = rule_80_20(df = cleanData, 
                                                    attr = 'Revenue', 
                                                    # value_index_1 = 2023,
                                                    # value_index_2 = 1
                                                    )

# For a  month
# print('De {:,} Markets para el mes '.format(numTotalMarket))
# print('{:,} Markets suplen el 80% de la demanda y '.format(numberMarketForDemand))
# print('{:,} Markets generan el 80% del Revenue'.format(numberMarketForRevenue))

# For every month
print('Para cada mes del ano se muestra cada una de las cantidades de markets que representan el 80% de la demanda:')
print(pd.concat([numMarketMonth,numberMarketForDemand['NumMarket']], axis=1) ,'\n')

print('Para cada mes del ano se muestra cada una de las cantidades de markets que representan el 80% de loa ingresos:')
print(pd.concat([numMarketMonth,numberMarketForRevenue['NumMarket']], axis=1) )

Para cada mes del ano se muestra cada una de las cantidades de markets que representan el 80% de la demanda:
    Year  MonthYear  Market  NumMarket
0   2022         12    1907        144
1   2023          1    1873        142
2   2023          2    1816        139
3   2023          3    1849        145
4   2023          4    1878        138
5   2023          5    1884        132
6   2023          6    1907        141
7   2023          7    1931        148
8   2023          8    1949        155
9   2023          9    1945        142
10  2023         10    1931        149
11  2023         11    1957        155
12  2023         12    1146        125 

Para cada mes del ano se muestra cada una de las cantidades de markets que representan el 80% de loa ingresos:
    Year  MonthYear  Market  NumMarket
0   2022         12    1907        133
1   2023          1    1873        127
2   2023          2    1816        122
3   2023          3    1849        138
4   2023          4    1878        12

In [ ]:
# Add histogram data
# df = cleanData.groupby(by=['WeekYear','Market'])['Bookings'].sum()
# df =  df.reset_index()
df = cleanData
x1 = df[df['Market']=='S46-S31']['Bookings']
x2 = df[df['Market']=='S31-S46']['Bookings']

fig = go.Figure()
fig = make_subplots(rows=1, cols=2, subplot_titles=("Histograma 1", "Histograma 2"))

fig.add_trace(go.Histogram(x=x1, name='Market S46-S31', marker=dict(line=dict(color='black', width=1))), row=1, col=1)
fig.add_trace(go.Histogram(x=x2, name='Market S46-S31', marker=dict(line=dict(color='black', width=1))), row=1, col=2)

#Actualizar el diseño del gráfico
fig.update_layout(title={
                        'text':'Histograma de Bookings para Market S46-S31', 
                        'x': 0.5, 
                        'xanchor':'center',
                        'font':{'color':'blue'}
                        },
                  xaxis_title='Bookings',
                  yaxis_title='Count')

#Ajustar los límites del eje x para agregar un zoom
fig.update_xaxes(range=[0, 60], row=1, col=1)
fig.update_xaxes(range=[0, 60], row=1, col=2)

#Mostrar el gráfico
fig.show()
# df.head()


In [ ]:
# Add histogram data
df = cleanData.groupby(by=['WeekYear', 'Market'])['Bookings'].sum().reset_index()
# df = cleanData
x1 = df[df['Market'] == 'S46-S31']['Bookings']
x2 = df[df['Market'] == 'S31-S46']['Bookings']

fig = go.Figure()

# Agregar ambos histogramas
fig.add_trace(go.Histogram(x=x1, name='Market S46-S31', marker=dict(color='rgba(255, 140, 0, 0.6)',line=dict(color='black', width=1))))
fig.add_trace(go.Histogram(x=x2, name='Market S31-S46', marker=dict(color='rgba(0, 134, 139, 0.3)',line=dict(color='black', width=1))))

# Actualizar el diseño del gráfico para que las barras se superpongan
fig.update_layout(title={
                        'text':'Histograma de Demanda para Market S46-S31 y S31-S46', 
                        'x': 0.5, 
                        'xanchor':'center',
                        'font':{'color':'rgba(0, 134, 139, 1)'}
                        },
                  xaxis_title='Bookings',
                  yaxis_title='Count',
                  bargap=0,  # Espacio entre las barras
                  barmode='overlay'  # Superponer las barras
                 )

# Ajustar los límites del eje x para que sean iguales en ambos histogramas
# fig.update_xaxes(range=[min(df['Bookings']), max(df['Bookings'])])
# fig.update_xaxes(range=[0, 40])
# Mostrar el gráfico
fig.show()

In [ ]:

df = cleanData[cleanData['Market']=='S46-S31'].sort_values(by='WeekYear',ascending=True)
yy = df.groupby(by='WeekYear').agg({'Revenue':sum,'Bookings':sum,})
yy = yy.reset_index()

fig = go.Figure(go.Scatter(
    x = yy['WeekYear'],
    y = yy['Bookings'],
    name='Demanda' 
))

fig.update_layout(
    title={
        'text': "Demand by WeekYear",
        'x': 0.5,  # Ajuste para centrar horizontalmente el título
        'xanchor': 'center',  # Ancla del título en el centro
        'font': {'color': 'blue'}
    },
    xaxis = dict(
        title="Week of the Year", 
        tickmode = 'linear',
        tick0 = yy['WeekYear'].min(),
        dtick = 1
    ),
    yaxis=dict(
        title="Demand"  # Nombre del eje y
    )
)
fig.show()


In [ ]:
df = pd.pivot_table(cleanData, values=['Bookings'], index='DayWeek', columns='HourDay', aggfunc={'Bookings':'sum'})
df = df.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

# Crear el heatmap
heatmap = go.Heatmap(
    z=df.values,
    x=df.columns.levels[1],
    y=df.index,
    hoverongaps=False,
    colorscale='Portland',  # Cambiar la escala de color (ejemplo: 'Viridis')
)

# Crear la figura y agregar el heatmap
fig = go.Figure(data=heatmap)

# Configurar el diseño del gráfico
fig.update_layout(
    title='Heatmap of Bookings by Day of Week and Hour of Day',
    xaxis=dict(
        title='Hour of Day',
        tickmode='array',
        tickvals=list(range(24)),  # Especificar todos los valores de x
        ticktext=[str(i) for i in range(24)]  # Etiquetas personalizadas para cada valor de x
    ),
    yaxis=dict(
        title='Day of Week'
    )
)

# Mostrar el gráfico
fig.show()

In [ ]:
# def rule_80_20(df: pd.DataFrame, attribute:str, percent = 80.0):
#     # Distribution of Market by [Demand]
#     marketFor = df.groupby(by=['Market'])[attribute].sum().sort_values(ascending=False)
#     marketFor = marketFor.reset_index()

#     marketFor['Porcent'] = (marketFor[attribute]/marketFor[attribute].sum())*100
#     marketFor['AcumulSum'] = marketFor['Porcent'].cumsum()

#     marketFor_80 = marketFor[marketFor['AcumulSum'] <= percent]
#     numberMarketFor_80 = marketFor_80['Market'].nunique()
    
#     return marketFor_80, numberMarketFor_80



# a, b= rule_80_20(cleanData, 'Revenue')
# a = a.reset_index()
# a[(a['Year']==2023) & (a['MonthYear']==1)]['Market'].nunique()

In [ ]:
# DataFrame dado
df = pd.DataFrame({
    'ano': [2022,2022,2022, 2030, 2030, 2030,2030,2030],
    'mes': ['11', '11', '11', '1','1', '2', '2', '2'],
    'market': ['X','z', 'Y', 'X', 'Y', 'X','y','z'],
    'valor': [10, 5, 20, 30, 40, 50, 15, 30]
})

# Calculamos la tabla pivote con los porcentajes
pivot_table = pd.pivot_table(df, index=['ano', 'mes', 'market'], values='valor', aggfunc='sum')
porcentajes = pivot_table.div(pivot_table.groupby(['ano', 'mes']).transform('sum')) * 100

porcentajes.index.get_level_values('ano')

# Calculamos la suma acumulada de los porcentajes por año y mes
suma_acumulada = porcentajes.groupby(['ano', 'mes']).cumsum()

# Filtramos los índices donde la suma acumulada sea menor que 80
indices_filtrados = suma_acumulada['valor'] < 80

# Obtenemos los índices de las filas filtradas
indices_filas_filtradas = indices_filtrados[indices_filtrados].index

# Filtramos el DataFrame por esos índices
df_filtrado = porcentajes.loc[indices_filas_filtradas].reset_index

print(indices_filtrados[indices_filtrados].index)

print(porcentajes)